In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import numpy as np
from torch.utils.data import random_split
from xgboost.sklearn import XGBClassifier


In [2]:
def load_data(filename):
    data = np.loadtxt(filename, delimiter=',', dtype=np.float32)
    x = data[:, 0:-2]
    y = data[:, [-1]] # after 3 months (-2) and 6 months (-1)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

In [3]:
# target to fix code


class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Sigmoid())
        self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        self.dropout = nn.Dropout(0.0001)
        #self.layer1 = nn.Sequential(nn.Linear(input_size, hidden_size), nn.Tanh())
        self.activation = torch.nn.ReLU()
        #self.dropout = nn.Dropout(0.1)
        #self.layer2 = nn.Sequential(nn.Linear(hidden_size, output_size), nn.Sigmoid())
        #self.dropout = nn.Dropout(0.1)
        #self.activation = torch.nn.ReLU()
        
        
        

        
    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.activation(x)
        
        
        return x # return model

In [4]:
x_train, y_train = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_training_dataset.csv')
x_test, y_test = load_data('/Users/withmocha/Library/CloudStorage/OneDrive-SejongUniversity/3학년 2학기/딥러닝개론/과제/과제1/hw1_test_dataset.csv')

In [5]:
y_train

tensor([[1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
      

In [6]:
x_test

tensor([[0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.2000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.2474, 0.5331, 0.1690, 0.5000, 0.7500, 0.4545, 0.9561, 0.2500, 0.3926,
         0.2303, 0.3000, 0.0556, 0.0000, 0.3043, 0.0500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 0.7000, 0.6111, 1.0000, 0.1652, 0.4800],
        [0.0042, 0.4179, 0.2535, 0.5000, 0.2500, 0.2727, 0.3986, 0.0000, 0.0000,
         0.2388, 0.6000, 0.5000, 1.0000, 0.1739, 0.7500],
        [0.3268, 0.3718, 0.3380, 0.5000, 0.5000, 0.5455, 0.5575, 0.2500, 0.3998,
         0.2061, 1.0000, 0.7222, 0.0000, 0.1739, 0.0000],
        [0.3208, 0.4467, 0.2676, 0.5000, 0.7500, 0.2727, 0.7489, 0.2812, 0.3559,
         0.1939, 1.0000, 0.7222, 0.0000, 0.3043, 0.7500],
        [0.0084, 0.5389, 0.0000, 1.0000, 0.2500, 0.1818, 0.6514, 0.0000, 0.0735,
         0.3152, 0.4000, 1.0000, 1.0000, 0.2609, 0.0000],
        [0.0168, 0.7176, 0.

In [7]:
x_train.shape

torch.Size([103, 15])

In [8]:


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test) # test set에는 transform만 사용하기


x_train=torch.Tensor(x_train)
x_test=torch.Tensor(x_test)

In [9]:
model = FNN(input_size=15, hidden_size=300, output_size=1)
criterion = nn.BCELoss()
#optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix
optimizer = optim.Adam(model.parameters(),lr=0.05, betas=(0.95, 0.999),eps=1e-08,weight_decay=0.0001) # target to fix

In [10]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1832, -0.0927,  0.2187,  ...,  0.0193, -0.2441, -0.1695],
        [-0.0393, -0.0585,  0.1273,  ...,  0.2495, -0.0517,  0.1151],
        [ 0.1935,  0.0901,  0.0007,  ..., -0.2292, -0.2221,  0.1040],
        ...,
        [-0.2336,  0.0458, -0.1678,  ..., -0.0899, -0.2148,  0.2039],
        [-0.0467, -0.1108, -0.1525,  ...,  0.2231,  0.0674,  0.1920],
        [ 0.0781, -0.1797,  0.1213,  ...,  0.2313,  0.0076,  0.1778]],
       requires_grad=True)
Parameter containing:
tensor([-0.1036, -0.2182,  0.0995,  0.0622,  0.2433,  0.2406,  0.0212,  0.0034,
        -0.0666,  0.2358,  0.1183, -0.0769, -0.1420,  0.2527, -0.1718, -0.1668,
        -0.2418, -0.1167, -0.1500,  0.2047,  0.1853,  0.1349, -0.1086, -0.0545,
         0.1201,  0.1644, -0.0702,  0.1048,  0.1385,  0.1090,  0.1360,  0.1028,
         0.1655, -0.0796,  0.1239, -0.0707, -0.1980,  0.1958,  0.1908,  0.1828,
        -0.0535, -0.0125,  0.2196, -0.0886,  0.0857,  0.0283,  0.1394,  0.1536,
        -0.0138

In [11]:
# assuming y is torch.Tensor

In [12]:
num_epochs = 1001

for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch % 200 == 0:
        print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item():.4f}')
        

with torch.no_grad(): 
    train_accuracy = (model(x_train).gt(0.5).float() == y_train).float().mean().item()
    test_accuracy = (model(x_test).gt(0.5).float() == y_test).float().mean().item()
    
    

print(f'\nTrain Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')
#Train Accuracy: 0.9709 -> start
#Test Accuracy: 0.9000 -> start

Epoch [0/1001], Loss: 0.7193
Epoch [200/1001], Loss: 0.1346
Epoch [400/1001], Loss: 0.0532
Epoch [600/1001], Loss: 0.0322
Epoch [800/1001], Loss: 0.0240
Epoch [1000/1001], Loss: 0.0197

Train Accuracy: 1.0000
Test Accuracy: 0.9500
